In [1]:
import pandas as pd
import numpy as np
import pathlib
import os

In [2]:
df_train = pd.read_csv("training_1000.nt",sep = ' ', header = None)
df_unseen = pd.read_csv("unseen_entities_1000.txt",sep = ' ', header = None)
df_val = pd.read_csv("validation_1000.nt",sep = ' ', header = None)
df_test = pd.read_csv("test_1000.nt",sep = ' ', header = None)
df_aux = pd.read_csv("auxiliary_1000.nt",sep = ' ', header = None)

In [3]:
print("df_train.shape",df_train.shape)
print("df_unseen.shape",df_unseen.shape)
print("df_val.shape",df_val.shape)
print("df_test.shape",df_test.shape)
print("df_aux.shape",df_aux.shape)

df_train.shape (129743, 4)
df_unseen.shape (1000, 1)
df_val.shape (5000, 4)
df_test.shape (1000, 4)
df_aux.shape (10529, 4)


In [4]:
def create_mappings(X):
    unique_ent = np.unique(np.concatenate((X[:, 0], X[:, 2])))
    unique_rel = np.unique(X[:, 1])
    ent_count = len(unique_ent)
    rel_count = len(unique_rel)
    rel_to_idx = dict(zip(unique_rel, range(rel_count)))
    ent_to_idx = dict(zip(unique_ent, range(ent_count)))
    return rel_to_idx, ent_to_idx

In [5]:
def to_idx(X, ent_to_idx=None, rel_to_idx=None):
    
    x_idx_s = np.vectorize(ent_to_idx.get)(X[:, 0])
    x_idx_p = np.vectorize(rel_to_idx.get)(X[:, 1])
    x_idx_o = np.vectorize(ent_to_idx.get)(X[:, 2])

    return np.dstack([x_idx_s, x_idx_p, x_idx_o]).reshape((-1, 3))

In [6]:
# def to_idx1(X, ent_to_idx=None, rel_to_idx=None):
#     x_idx_s = np.vectorize(ent_to_idx.get)(X[:, 0])
#     return np.dstack([x_idx_s]).reshape((-1, 1))

In [7]:
def load_from_csv(folder_name, file_name, sep=' ', header=None):
    df = pd.read_csv(os.path.join(path, file_name),
                     sep=sep,
                     header=header,
                     names=None,
                     dtype=str)

    df = df.drop_duplicates()
    return df.as_matrix()

In [9]:
path = os.getcwd()
train = load_from_csv(path,'training_1000.nt')
aux = load_from_csv(path,'auxiliary_1000.nt')
val = load_from_csv(path,'validation_1000.nt')
# unseen = load_from_csv(path,'unseen_entities_1000.txt')
test = load_from_csv(path,'test_1000.nt')

/Users/gurpreet.ag.singh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [10]:
vocab = np.concatenate((train,aux,val))

In [11]:
rel_to_idx, ent_to_idx = create_mappings(vocab)

train = to_idx(train , ent_to_idx, rel_to_idx)
aux = to_idx(aux, ent_to_idx, rel_to_idx)
val = to_idx(val, ent_to_idx, rel_to_idx)
test = to_idx(test, ent_to_idx, rel_to_idx)

In [13]:
print("train.shape",train.shape)
print("val.shape",val.shape)
print("test.shape",test.shape)
print("aux.shape",aux.shape)

train.shape (129743, 3)
val.shape (5000, 3)
test.shape (1000, 3)
aux.shape (10529, 3)


In [14]:
path = os.getcwd()
pathlib.Path(path+'/Combined_Seralised').mkdir(parents=True, exist_ok=True)
print("A New Directory is created, named: 'Combined_Seralised'")

A New Directory is created, named: 'Combined_Seralised'


In [15]:
path

'/Users/gurpreet.ag.singh/Desktop/dbpedia_1k_s (1)/schema and not schama'

In [26]:
train

array([[ 5952,   305, 32343],
       [ 9267,   293, 63482],
       [24794,   269, 72969],
       ...,
       [29754,   359, 44408],
       [18976,   380, 75797],
       [43502,    47, 43518]])

In [16]:
df_train = pd.DataFrame(train,index=None, columns=None)
df_aux = pd.DataFrame(aux,index=None, columns=None)
df_val = pd.DataFrame(val,index=None, columns=None)
df_test = pd.DataFrame(test,index=None, columns=None)

In [17]:
path = os.getcwd()+'/Combined_Seralised/'
df_train.to_csv(path+'train',sep='\t', header=False, index=False, encoding = "utf-8")
df_aux.to_csv(path+'aux',sep='\t', header=False, index=False, encoding = "utf-8")
df_val.to_csv(path+'val',sep='\t', header=False, index=False, encoding = "utf-8")
df_test.to_csv(path+'test',sep='\t', header=False, index=False, encoding = "utf-8")

print("All the seralised files are now saved inside Combined_Seralised folder")

All the seralised files are now saved inside Combined_Seralised folder


## ADDING TRUE COLUMN OF 1 = TRUE in TEST AND VALIDATE

In [19]:
path

'/Users/gurpreet.ag.singh/Desktop/dbpedia_1k_s (1)/schema and not schama/Combined_Seralised/'

In [20]:
val = np.genfromtxt(path+"val", dtype='int')
test = np.genfromtxt(path+"test", dtype='int')

In [21]:
print("val shape:", val.shape)
print("test shape:", test.shape)

val shape: (5000, 3)
test shape: (1000, 3)


In [22]:
val_ones = np.ones((5000,1), dtype=int)
test_ones = np.ones((1000,1), dtype=int)

In [23]:
val = np.concatenate((val,val_ones),axis=1)
test = np.concatenate((test,test_ones),axis=1)

In [25]:
np.savetxt('val_c', val, delimiter="\t", fmt="%d") 
np.savetxt('test_trues', test, delimiter="\t", fmt="%d")